In [2]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from src import deepTarget_utils as bio_utils
import re
import src.RNA
from  src.RNA import cofold
from src import my_deepTarget_utils as utils

In [3]:
poss_site_level = "../data/13059_2014_500_MOESM1_ESM=site-level human miRNA-target site pairs from miRecords=.csv"
poss_gene_level = "../data/13059_2014_500_MOESM2_ESM=UTR-level human miRNA-gene pairs from miRecords and miRTarBase=.csv"

In [4]:
data = pd.read_csv(poss_site_level)

In [5]:
mirna_fasta_file = "../data/miRNA.fasta"
mrna_fasta_file = "../data/3UTR.fasta"
mirna_ids, mirna_seqs, mrna_ids, mrna_seqs = bio_utils.read_fasta(mirna_fasta_file, mrna_fasta_file)

In [13]:
SEED_START = 1
SEED_SIZE = 6
CTS_SIZE = 40
wc_pairs = {'A': 'U', 'U': 'A', 'G': 'C', 'C': 'G'}

In [10]:
data["seq"] = data.mRNA_ID.apply(lambda x: mrna_seqs[mrna_ids.index(x)])
data["miseq"] = data.miR_ID.apply(lambda x: mirna_seqs[mirna_ids.index(x)])

In [15]:
# very heavy to run
end = len(data) # if you have all the power
end = 10 # can take 10 minutes
# TODO make multiprocess
res = []
for index in range(0, end):
    datum = data.loc[index]
    
    end_index = datum.End_position
    mirna = datum.miseq
    mrna = datum.seq
    if end_index not in bio_utils.find_candidate(mirna, mrna):
        continue
    
    mirna_seed = mirna[SEED_START:SEED_START+SEED_SIZE]
    mirna_so = mirna[:SEED_START]
    mirna_rest = mirna[SEED_START+SEED_SIZE:]

    
    mrna_target = mrna[end_index-SEED_SIZE-1:end_index-1]
    mrna_end = mrna[end_index-1:]
    mrna_start = mrna[:end_index-SEED_SIZE-1]
    combined_mfe = (cofold(mirna_seed+"&"+mrna_target)[1]
         + cofold(mrna_end+"&"+mrna_start)[1]
         + cofold(mirna_rest+"&"+mirna_so)[1])
    mfe = cofold(mirna)[1] + cofold(mrna)[1]
    res.append([combined_mfe, mfe])

In [16]:
# you can check res
res

[[-294.8999876976013, -292.6000061035156],
 [-495.4000062942505, -489.6000061035156]]